In [1]:
import numpy as np
import cv2
from scipy.ndimage import center_of_mass

In [2]:
ORANGE_AREA_DEPTH = 230
ORANGE_AREA_HEIGHT_PX = 72
ORANGE_AREA_HEIGHT_ACTUAL = 0.36

In [5]:
print(depth)

[[  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 ...
 [252.15005 252.07951 252.00899 ... 183.76816 183.49728 183.22647]
 [251.98883 251.98016 251.97147 ... 183.94276 183.92487 183.907  ]
 [251.8276  251.8808  251.93398 ... 184.1174  184.35251 184.5876 ]]


In [7]:
print(depth.shape)

(1080, 1920)


In [3]:
import cv2
import numpy as np
from scipy.ndimage import center_of_mass

# Load the saved arrays
vessel_mask = np.load('./outputs/2024_1126_143639F_trimmed/object_detection/mask_arrays/2024_1126_143639F_trimmed_mask_array_id_1_frame_355.npy')
depth = np.load('./outputs/2024_1126_143639F_trimmed/depth_estimation/depth_arrays/2024_1126_143639F_trimmed_depth_array_id_1_frame_355.npy')

# Load the original image
img = cv2.imread('outputs/2024_1126_143639F_trimmed/tracked_vessels/2024_1126_143639F_trimmed_vessel_detected_shot_id_1_frame_355.png')

# Measure the height in pixels
rows, _ = np.where(vessel_mask > 0)
vessel_height_px = np.max(rows) - np.min(rows)

# Find the center of the boat
center_y, center_x = center_of_mass(vessel_mask)  # Center in (y, x) format

print(f"Vessel center: ({center_x}, {center_y})")
print(f"Vessel height (pixels): {vessel_height_px}")

# Normalize depth map
depth_normalized = (depth - depth.min()) / (depth.max() - depth.min()) * 255
depth_colored = cv2.applyColorMap(depth_normalized.astype(np.uint8), cv2.COLORMAP_JET)

# Create an overlay with the mask
mask_overlay = cv2.addWeighted(depth_colored, 0.4, cv2.cvtColor((vessel_mask * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR), 0.6, 0)

# Create an overlay with the original image (no mask)
image_overlay = cv2.addWeighted(img, 0.6, depth_colored, 0.4, 0)

# Combine the heatmap, mask overlay, and image overlay side-by-side
combined_image = cv2.hconcat([depth_colored, mask_overlay, image_overlay])

# Save the combined image
output_path = './vessel_heatmap_combined_with_original.png'
cv2.imwrite(output_path, combined_image)

print(f"Combined heatmap with original image saved to: {output_path}")


Vessel center: (1312.809682878052, 623.4211370678856)
Vessel height (pixels): 135
Combined heatmap with original image saved to: ./vessel_heatmap_combined_with_original.png


In [55]:
# Convert boat_mask to boolean
vessel_mask_boolean = vessel_mask > 0  # Ensure the mask is boolean (True/False)

# Calculate rows_with_mask using the original mask
rows_with_mask = np.any(vessel_mask_boolean, axis=1)

# Get the depth values corresponding to the mask
vessel_depth_values = depth_normalized[rows_with_mask, :]
vessel_depth_average = vessel_depth_values[vessel_mask_boolean[rows_with_mask, :]].mean()

# Get the depth at the boat center
vessel_center_depth = depth_normalized[int(center_y), int(center_x)]

print(f"Depth of boat at center: {vessel_center_depth}")
print(f"Depth of boat (average): {vessel_depth_average}")

Depth of boat at center: 36.0540771484375
Depth of boat (average): 33.78672790527344


In [49]:
# Formula -> vessel_height_actual = vessel_height_px * (oa_height_actual/oa_height_px) * (oa_depth/vessel_depth)
oa_height_actual = ORANGE_AREA_HEIGHT_ACTUAL
oa_height_px = ORANGE_AREA_HEIGHT_PX
oa_depth = ORANGE_AREA_DEPTH
vessel_depth = vessel_center_depth
vessel_height_px = vessel_height_px
vessel_height_actual = vessel_height_px * (oa_height_actual/oa_height_px) * (oa_depth/vessel_depth)
print(f"Estimated Vessel Height: {vessel_height_actual}")

Estimated Vessel Height: 4.306031724534882
